In [13]:
from spotipy.oauth2 import SpotifyOAuth
import spotipy

# Set up Spotify API credentials
client_id = '14aca921d4d342e691b7c93091e62281'
client_secret = '166636d6e0ff43bc879ea9270ac34295'
redirect_uri = 'https://example.com/callback'  # Matches the URI in the Spotify dashboard
scope="user-read-private, user-library-read, user-read-recently-played, user-top-read, user-read-email, playlist-read-private, playlist-read-collaborative"

# Initialize SpotifyOAuth for manual handling of the redirect URL
sp_oauth = SpotifyOAuth(
    client_id=client_id,
    client_secret=client_secret,
    redirect_uri=redirect_uri,
    scope=scope
)

# Get the authorization URL
auth_url = sp_oauth.get_authorize_url()
print("Go to the following URL to authorize the app:")
print(auth_url)

# Manually copy and paste the redirected URL
redirect_response = input("Paste the full redirect URL here: ")

# Extract the authorization code from the URL
code = sp_oauth.parse_response_code(redirect_response)

# Fetch the access token using the authorization code
token_info = sp_oauth.get_access_token(code)
access_token = token_info['access_token']

# If the token is expired, refresh it
if sp_oauth.is_token_expired(token_info):
    token_info = sp_oauth.refresh_access_token(token_info['refresh_token'])
    access_token = token_info['access_token']

# Initialize Spotify client with the access token
sp = spotipy.Spotify(auth=access_token)


Go to the following URL to authorize the app:
https://accounts.spotify.com/authorize?client_id=14aca921d4d342e691b7c93091e62281&response_type=code&redirect_uri=https%3A%2F%2Fexample.com%2Fcallback&scope=+playlist-read-collaborative++playlist-read-private++user-library-read++user-read-email++user-read-recently-played++user-top-read+user-read-private


C:\Users\TASHA\AppData\Local\Temp\ipykernel_6132\3509867592.py:30: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token_info = sp_oauth.get_access_token(code)


In [2]:
# Test the API connection
results = sp.search(q='Sauti Sol', limit=5)
for idx, track in enumerate(results['tracks']['items']):
    print(f"{idx + 1}: {track['name']} by {track['artists'][0]['name']}")

1: Still the one by Sauti Sol
2: Feel My Love by Sauti Sol
3: Oversized T-shirt by Matata
4: Insecure by Sauti Sol
5: Sober by Sauti Sol


In [9]:
# Test API connection
result = sp.search(q="pop", type="playlist", limit=1)
print(result)

{'playlists': {'href': 'https://api.spotify.com/v1/search?offset=0&limit=1&query=pop&type=playlist', 'limit': 1, 'next': 'https://api.spotify.com/v1/search?offset=1&limit=1&query=pop&type=playlist', 'offset': 0, 'previous': None, 'total': 103, 'items': [{'collaborative': False, 'description': '', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/5Mt7LZirLgrfr90njvoNCw'}, 'href': 'https://api.spotify.com/v1/playlists/5Mt7LZirLgrfr90njvoNCw', 'id': '5Mt7LZirLgrfr90njvoNCw', 'images': [{'height': 640, 'url': 'https://mosaic.scdn.co/640/ab67616d00001e02c596d2af5b5c92628563952bab67616d00001e02e24f91c2ac3eea3d398a1a8dab67616d00001e02f4a8580b0594a02f5706b4d9ab67616d00001e02fef15f43f9f17c7caf381fe2', 'width': 640}, {'height': 300, 'url': 'https://mosaic.scdn.co/300/ab67616d00001e02c596d2af5b5c92628563952bab67616d00001e02e24f91c2ac3eea3d398a1a8dab67616d00001e02f4a8580b0594a02f5706b4d9ab67616d00001e02fef15f43f9f17c7caf381fe2', 'width': 300}, {'height': 60, 'url': 'https://mosaic.sc

In [ ]:
import time
import json
import re  # To sanitize file names

    # Function to get playlist IDs by genre
def get_playlist_ids_by_genre(genre, limit=5, sleep_time=2, max_pages=3):
    playlist_ids = []
    results = sp.search(q=genre, type='playlist', limit=limit)

    print(f"Fetched results for {genre}: {results}")

    # Check if 'playlists' key exists in results and has 'items'
    if results and results.get('playlists') and results['playlists'].get('items'):
        for playlist in results['playlists']['items']:
            if playlist and playlist.get('id'):
                playlist_ids.append(playlist['id'])

    page_count = 0  # Track how many pages we've fetched

    # Handle pagination: check if there are more playlists
    while results and results.get('playlists') and results['playlists'].get('next'):
        if page_count >= max_pages:  # Stop after fetching 'max_pages' times
            print(f"Reached max page limit ({max_pages}) for {genre}. Stopping pagination.")
            break

        print(f"Fetching next page for {genre}...")  # Debugging

        results = sp.next(results['playlists'])
        if results and results.get('playlists') and results['playlists'].get('items'):
            for playlist in results['playlists']['items']:
                if playlist and playlist.get('id'):
                    playlist_ids.append(playlist['id'])

        # Add sleep time after each API call to avoid rate limiting
        time.sleep(sleep_time)
        page_count += 1  # Increment page count

    print(f"Fetched {len(playlist_ids)} playlists for {genre}")  # Debugging
    
    return playlist_ids[:limit]

# Function to sanitize file names
def sanitize_filename(filename):
    return re.sub(r'[^\w\-_.]', '_', filename)

# List of genres to scrape playlists for
genres = ['reggae', 'house', 'rap', 'indie', 'R_B', 'soul', 'amapiano', 'afrobeats', 'alternative']

# Scrape playlist IDs for each genre
for genre in genres:
    print(f"Fetching playlists for genre: {genre}")
    playlist_ids = get_playlist_ids_by_genre(genre, limit=5)
    
    # Sanitize the file name
    safe_genre_name = sanitize_filename(genre)
    
    # Save playlist IDs to a JSON file
    with open(f"{safe_genre_name}_playlists.json", "w", encoding="utf-8") as file:
        json.dump(playlist_ids, file, indent=4)
    
    print(f"Saved {len(playlist_ids)} playlist IDs to {safe_genre_name}_playlists.json")
    time.sleep(2)


Fetching playlists for genre: reggae
Fetched results for reggae: {'playlists': {'href': 'https://api.spotify.com/v1/search?offset=0&limit=5&query=reggae&type=playlist', 'limit': 5, 'next': 'https://api.spotify.com/v1/search?offset=5&limit=5&query=reggae&type=playlist', 'offset': 0, 'previous': None, 'total': 902, 'items': [None, {'collaborative': False, 'description': 'Best played on shuffle🔥', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/45WUjR60no7LI4fKhby1b3'}, 'href': 'https://api.spotify.com/v1/playlists/45WUjR60no7LI4fKhby1b3', 'id': '45WUjR60no7LI4fKhby1b3', 'images': [{'height': 640, 'url': 'https://mosaic.scdn.co/640/ab67616d00001e021a4388152711e117ed532fb8ab67616d00001e02234305971a0967afe6f95146ab67616d00001e02d21ddf0dfe22eb2c145ff87bab67616d00001e02ed2292577d3da7992c59fc07', 'width': 640}, {'height': 300, 'url': 'https://mosaic.scdn.co/300/ab67616d00001e021a4388152711e117ed532fb8ab67616d00001e02234305971a0967afe6f95146ab67616d00001e02d21ddf0dfe22eb2c145ff8

In [ ]:
import time
import json

# Function to get song IDs from a playlist
def get_song_ids_from_playlist(playlist_id, limit=100):
    song_ids = []
    results = sp.playlist_tracks(playlist_id, limit=limit)
    for item in results['items']:
        song = item.get('track')
        if song:  # Check if the track is not None
            song_ids.append(song['id'])
    return song_ids

# List of genres to process

genres = ['reggae', 'house', 'rap', 'indie', 'R_B', 'soul', 'amapiano', 'afrobeats', 'alternative']

for genre in genres:
    try:
        # Load playlist IDs from the JSON file for the current genre
        with open(f"{genre}_playlists.json", "r") as file:
            playlist_ids = json.load(file)
        
        all_song_ids = set()  # Use a set to avoid duplicates

        for playlist_id in playlist_ids:
            print(f"Fetching songs from playlist: {playlist_id}")
            try:
                song_ids = get_song_ids_from_playlist(playlist_id)
                all_song_ids.update(song_ids)
            except Exception as e:
                print(f"Error fetching songs from playlist {playlist_id}: {e}")
            time.sleep(1)  # Avoid rate limiting

        # Save song IDs to a JSON file
        with open(f"{genre}_songs.json", "w") as file:
            json.dump(list(all_song_ids), file)

        print(f"Saved {len(all_song_ids)} song IDs to {genre}_songs.json")
    
    except FileNotFoundError:
        print(f"File {genre}_playlists.json not found. Skipping genre.")
    except Exception as e:
        print(f"Error processing genre {genre}: {e}")


Fetching songs from playlist: 5tSbO6M5AT7r91ytDd4nNP
Fetching songs from playlist: 4gpg7XaGy6kHzZBklmFy7H
Fetching songs from playlist: 37Aby3TQuKu67YJQEQwvVh
Fetching songs from playlist: 2T3BSpqN34Z4sppHDNWoeE
Fetching songs from playlist: 4h9dvquufDVDyGuYRHeu73
Saved 365 song IDs to R_B_songs.json


In [14]:
import time
import json
import pandas as pd
import random
from spotipy.exceptions import SpotifyException

# Function to fetch basic song details with retry logic for rate limits
def fetch_song_details(song_id):
    while True:
        try:
            song = sp.track(song_id)
            return {
                'song_id': song['id'],
                'name': song['name'],
                'artist': song['artists'][0]['name'],
                'album': song['album']['name'],
                'popularity': song['popularity']
            }
        except SpotifyException as e:
            if e.http_status == 429:  # Rate limit error
                retry_after = int(e.headers.get("Retry-After", 5))
                print(f"Rate limit reached. Retrying after {retry_after} seconds.")
                time.sleep(retry_after)
            else:
                print(f"Error fetching details for song {song_id}: {e}")
                return None
        except Exception as e:
            print(f"Unexpected error for song {song_id}: {e}")
            return None


# Genre to process (only hip-hop as per your request)
genre = 'alternative'

try:
    # Load song IDs from the corresponding JSON file
    with open(f"{genre}_songs.json", "r") as file:
        song_ids = json.load(file)

    # Optionally limit the dataset to avoid excessive API calls
    sampled_song_ids = random.sample(song_ids, min(len(song_ids), 500))  # Limit to 500 songs

    # Fetch song details
    all_song_details = []
    for song_id in sampled_song_ids:
        print(f"Fetching details for song: {song_id}")
        song_details = fetch_song_details(song_id)
        if song_details:
            all_song_details.append(song_details)
        time.sleep(1)  # Wait 1 second between requests

    # Save song details to a CSV file
    df_details = pd.DataFrame(all_song_details)
    df_details.to_csv(f"{genre}_song_details.csv", index=False)
    print(f"Saved song details to {genre}_song_details.csv")


except FileNotFoundError:
    print(f"File not found for genre: {genre}. Ensure the JSON file exists.")
except Exception as e:
    print(f"Error processing genre {genre}: {e}")


Fetching details for song: 76VkZLP9FsL1VCpWTuzSdE
Fetching details for song: 4eSGSqP2TZvvX0kadZZttM
Fetching details for song: 48EjSdYh8wz2gBxxqzrsLe
Fetching details for song: 5OMNVzNkXIs6HP4tnExf5m
Fetching details for song: 754kgU5rWscRTfvlsuEwFp
Fetching details for song: 5bRJYxFMTuB2eT22Fd92lZ
Fetching details for song: 0tLIjCdtDDyyUeDGZ8iuwM
Fetching details for song: 7oHFFLFlLnEtxnxAricfn3
Fetching details for song: 1UcWNtdPvWzKBsley8vixH
Fetching details for song: 4foFLTIhapOuM8ORu1OqYV
Fetching details for song: 59acp1OhcvxwVBwQJBYKuX
Fetching details for song: 5OVqtLp71md1LFD3mucPHc
Fetching details for song: 4W9FiD4WNH22AUay8emLRF
Fetching details for song: 1r0R3msnUT2OytxsbkVbBP
Fetching details for song: 2AhAhd9BiyydtKq2hMdb7Q
Fetching details for song: 1XUrsaeZmnOuUsFCfSMCxM
Fetching details for song: 11yZHrCVjzHX9j8wxmwGEu
Fetching details for song: 3ZOtnNjv19m7EZwnSQ2TIM
Fetching details for song: 5MAA5ylLKqlawCsQdQLGlF
Fetching details for song: 5H8VsuRaxPKSSYxmmmuS04
